# IEOR 4571 - Personalization - Final Project

#### Team members: 
Name, UNI/email, Github ID
* Megala Kannan, msk2245@columbia.edu, thisismeg
* Hojin Lee, hl3328@columbia.edu, hjlee9295
* Jung Ah Shin, js5569@columbia.edu, juliajungahshin
* Tiffany Zhu, tz2196@columbia.edu, tlzhu19


# TOC:
* [1. Introduction](#1)
* [2. Data Exploration](#2)
* [3. Modeling](#3)
    * [3.1 Baseline Models](#3-1)
        * [3.1.1 Learned Bias Baseline Model](#3-1-1)
        * [3.1.2 Collaborative Filtering Baseline Model](#3-1-2)
    * [3.2 Collective Matrix Factorization](#3-2)
* [4. Evaluation](#4)
    * [4.1 Accuracy](#4-1)
    * [4.2 Coverage](#4-2)
* [5. Conclusion](#5)


# 1. Introduction <a class="anchor" id="1"></a>

# 2. Data Exploration <a class="anchor" id="2"></a>

In [27]:
import pandas as pd
import json
from tqdm import tqdm

import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')

import itertools
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [22]:
def convert_json_to_df(path, file_name, column_names):
    '''
    converts the json file into a pandas dataframe
    path: string of path
    file_name: string of json file name
    column_names: the names of the json file attributes (used as the dataframe column names)
    returns: dataframe
    '''
    line_count = len(open(path + file_name).readlines())    
    columns_dict = {name: [] for name in column_names}

    with open(path + file_name) as f:
        for line in tqdm(f, total=line_count):
            blob = json.loads(line)
            
            for key in column_names:
                columns_dict[key].append(blob[key])
    
    return pd.DataFrame(columns_dict)

In [23]:
path = "/Users/tiffanyzhu/Downloads/yelp_dataset/"
file_name = "review.json"

In [4]:
# review.json
ratings = convert_json_to_df(path, file_name, ['user_id', 'business_id', 'stars', 'date'])

user_counts = ratings["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()

100%|██████████| 6685900/6685900 [01:21<00:00, 82502.81it/s]


In [5]:
len(active_users)

286130

In [6]:
ratings.rename(columns={'stars': 'rating'}, inplace=True)
ratings.head()

,user_id,business_id,rating,date
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38


In [7]:
import random

# take random subset of active users
n = len(active_users)
subset_active_users = random.sample(active_users, round(n * 0.2))

# filter the ratings df by the subset of active users
active_user_ratings = ratings.loc[ratings['user_id'].isin(subset_active_users)]

In [8]:
len(active_user_ratings)

895298

In [9]:
active_user_ratings2 = active_user_ratings.sort_values('date')
actual_X = active_user_ratings2.groupby(['user_id'], as_index=False).apply(lambda x: x.iloc[:-1])
actual_y = active_user_ratings2.groupby(['user_id'], as_index=False).apply(lambda x: x.iloc[-1])

In [10]:
actual_X.head()

user_id             business_id  rating  \
0 5077203  --0kuuLmuYBe3Rmu0Iycww  sxPwFSLoW7xx1tWgNZ-p6g     5.0   
  6081103  --0kuuLmuYBe3Rmu0Iycww  6TBfgiKpP-VWtKuM-IwR0Q     1.0   
  4996945  --0kuuLmuYBe3Rmu0Iycww  loEwm40TwkQeEu3zYvU7RQ     4.0   
  3338704  --0kuuLmuYBe3Rmu0Iycww  VaiYxIUfHIfYfwYgOupjMA     4.0   
  3298263  --0kuuLmuYBe3Rmu0Iycww  ev1SC6q8AolQWix0n577sg     2.0   

                          date  
0 5077203  2013-08-26 23:07:49  
  6081103  2013-09-06 06:20:54  
  4996945  2013-09-12 00:27:27  
  3338704  2013-10-03 17:19:39  
  3298263  2013-11-11 20:35:14

In [11]:
actual_y.head()

,user_id,business_id,rating,date
0,--0kuuLmuYBe3Rmu0Iycww,PYe_FDw6QTbTf66WcGE_tw,2.0,2014-04-21 16:58:28
1,--2HUmLkcNHZp0xw6AMBPg,KW9RNyBPmc77f9FsO92qYw,5.0,2018-10-04 02:02:28
2,--4q8EyqThydQm-eKZpS-A,rcaPajgKOJC2vo_l3xa42A,3.0,2012-06-17 16:59:06
3,--7gjElmOrthETJ8XqzMBw,UxWH8zRYIBgs6Q2oykvRdw,4.0,2018-05-24 21:19:54
4,--Br-QsbO9ad5GbZxVGxaw,x6PA-2j7LpZAYFo2VojmQQ,2.0,2016-08-24 14:55:34


In [6]:
# can do the same for business.json, user.json, tip.json 
# for metadata info see https://www.yelp.com/dataset/documentation/main

In [24]:
# business.json
# todo: how to add 'attributes'?
'''
"attributes": {
        "RestaurantsTakeOut": true,
        "BusinessParking": {
            "garage": false,
            "street": true,
            "validated": false,
            "lot": false,
            "valet": false
        },
'''

businesses = convert_json_to_df(path, 'business.json', ['business_id', 'city', 'state', 'stars', 
                                                        'review_count', 'is_open', 'attributes', 
                                                        'categories', 'hours'])

100%|██████████| 192609/192609 [00:05<00:00, 35914.14it/s]


In [18]:
businesses.head()

,business_id,city,state,stars,review_count,categories
0,1SWheh84yJXfytovILXOAQ,Phoenix,AZ,3.0,5,"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Mississauga,ON,2.5,128,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Charlotte,NC,4.0,170,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Goodyear,AZ,5.0,3,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Charlotte,NC,4.0,4,"Plumbing, Shopping, Local Services, Home Servi..."


In [13]:
# user.json
users = convert_json_to_df(path, 'user.json', ['user_id', 'review_count', 'friends', 'useful', 
                                               'funny', 'cool', 'fans', 'elite', 'average_stars', 
                                               'compliment_hot', 'compliment_more', 'compliment_profile',
                                               'compliment_cute', 'compliment_list', 'compliment_note',
                                               'compliment_plain', 'compliment_cool', 'compliment_funny',
                                               'compliment_writer', 'compliment_photos'
                                              ])

100%|██████████| 1637138/1637138 [00:35<00:00, 46222.22it/s]


In [20]:
users.head()

,user_id,review_count,friends,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,95,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",84,17,25,5,"2015,2016,2017",4.03,2,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,33,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",48,22,16,4,,3.63,1,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,16,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",28,8,10,0,,3.71,0,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,17,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",30,4,14,5,,4.85,1,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,361,"mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",1114,279,665,39,"2015,2016,2017,2018",4.08,28,1,0,0,1,16,57,80,80,25,5


In [14]:
# tip.json
tips =  convert_json_to_df(path, 'tip.json', ['text', 'date', 'compliment_count', 'business_id', 'user_id'])

100%|██████████| 1223094/1223094 [00:09<00:00, 122404.61it/s]


In [29]:
photos = convert_json_to_df(path, 'photo.json', ['photo_id', 'business_id'])

100%|██████████| 200000/200000 [00:01<00:00, 194952.47it/s]


# 3. Modeling <a class="anchor" id="3"></a>

## 3.1 Baseline Model <a class="anchor" id="3-1"></a>

### 3.1.1 Learned Bias Baseline Model <a class="anchor" id="3-1-1"></a>

In [96]:
def find_average_user_ratings(ratings_df):
    ratings_series = ratings_df.groupby('user_id')['rating'].mean()
    return pd.DataFrame({'user_id':ratings_series.index, 'rating':ratings_series.values})

average_user_ratings = find_average_user_ratings(actual_X)

In [97]:
def find_average_business_ratings(ratings_df):    
    ratings_series = ratings_df.groupby('business_id')['rating'].mean()
    return pd.DataFrame({'business_id':ratings_series.index, 'rating':ratings_series.values})

average_business_ratings = find_average_business_ratings(actual_X)

In [99]:
# average_user_ratings.get(average_user_ratings.index[0])
average_user_ratings.head()

,user_id,rating
0,--BumyUHiO_7YsHurb9Hkw,3.885246
1,--HOeLECewlqBqvFhQYUVA,4.200000
2,--NIc98RMssgy0mSZL3vpA,4.000000
3,--XpHLy__U2RF_dkWCmmjw,4.250000
4,--YhjyV-ce1nFLYxP49C5A,3.402299


In [100]:
average_business_ratings.head()

,business_id,rating
0,--1UhMGODdWsrMastO9DZw,4.142857
1,--6MefnULPED_I942VcFNA,2.909091
2,--7zmmkVg-IMGaXbuVd0SQ,3.500000
3,--9QQLMTbFzLJ_oT-ON3Xw,3.000000
4,--9e1ONYQuAa-CB_Rrw7Tw,4.177778


In [124]:
# avg_business_ratings_dict = dict(zip(average_business_ratings.business_id, average_business_ratings.rating))
((average_user_ratings['rating'] + average_user_ratings['rating']) / 10).values

array([0.77704918, 0.84      , 0.8       , ..., 0.65882353, 0.6       ,
       0.85      ])

In [134]:
'PssNPwuSuOjHAx5WGhgbyg' in average_business_ratings['business_id']

False

In [140]:
actual_y

,user_id,business_id,rating,date
0,--BumyUHiO_7YsHurb9Hkw,0L3CAgRf8wuH5MjNwJdivg,3.0,2018-10-20 17:56:22
1,--HOeLECewlqBqvFhQYUVA,PssNPwuSuOjHAx5WGhgbyg,1.0,2018-03-26 07:55:42
2,--NIc98RMssgy0mSZL3vpA,SoUlHQO5jZf2XFHr8g2CTA,5.0,2018-03-05 21:43:05
3,--XpHLy__U2RF_dkWCmmjw,PvzD0DBJA6qRDF06hdUjKQ,4.0,2018-02-07 19:37:47
4,--YhjyV-ce1nFLYxP49C5A,Yh0ihCj3sG4bcgu3z6714g,4.0,2018-05-17 21:49:02
...,...,...,...,...
57221,zzW1NTuShpLFy-4zPFbggw,vsFFbN71ehRCp46KeR5RdQ,5.0,2018-09-09 02:50:09
57222,zza-gQ2jzjAAJkalUHuO6Q,1ylgrtS3iSNaf3V68kw4bw,5.0,2018-05-30 01:22:23
57223,zzjQuRnY_Kgr_zaw6A-ACQ,dQrDogW_V4fB83vTTAP5eA,4.0,2018-09-17 19:33:25
57224,zzltQbbtriSWWdNINMzJUw,dNMjWmRVUIep_NdXNSJRTA,5.0,2018-06-26 23:32:09


In [143]:
# ratings_df - for each user need to remove its last review

def predict_learned_bias(actual_ratings, average_user_ratings, average_business_ratings):

    avg_business_ratings_dict = dict(zip(average_business_ratings.business_id, average_business_ratings.rating))
    
    def find_business_avg(row):
        #todo: what if business we are trying to predict was not seen? give it 3 (out of 5)?
        return avg_business_ratings_dict.get(row['business_id'], 4)  
    
    avg_business_ratings = actual_ratings.apply(find_business_avg, axis=1)
    print('avg_business_ratings created!')
    
    # we are assuming that the order is the same in each avg_business_ratings and average_user_ratings
    predicted_rating = (avg_business_ratings + average_user_ratings['rating']) / 2
    print('predicted_rating created!')
    
    actual_ratings['predicted_rating'] = predicted_rating.values
    
    return actual_ratings
    
basine_line_predictions = predict_learned_bias(actual_y, average_user_ratings, average_business_ratings)

avg_business_ratings created!
predicted_rating created!


In [155]:
import numpy as np

N = len(basine_line_predictions)
baseline_rmse = np.sum(np.sqrt(((basine_line_predictions['predicted_rating'] - basine_line_predictions['rating']) / N) ** 2))

In [156]:
baseline_rmse

1.1332834326500296

### 3.1.2 Collaborative Filtering Baseline Model <a class="anchor" id="3-1-1"></a>

## 3.2 Collective Matrix Factorization <a class="anchor" id="3-2"></a>

In [25]:
active_user_only_ratings_df = ratings[ratings['user_id'].isin(active_users)]

sample_size = [100]

for s in sample_size:
    sampleUID = active_user_only_ratings_df['user_id'].unique()[np.random.randint(active_user_only_ratings_df['user_id'].unique().shape[0], size=s)]
    active_user_only_ratings_df_sample = active_user_only_ratings_df[active_user_only_ratings_df['user_id'].isin(sampleUID)]



In [30]:
base_df = active_user_only_ratings_df_sample

#sparse alert - hopefully with bigger dataset, we will see some weird ratings..
real_average = base_df[['business_id','rating']].groupby('business_id').mean().reset_index()
real_average.rename(columns={"rating": "average_business_rating"}, inplace=True)

#Average ratings for business added
base_df = base_df.merge(real_average, how='left', on='business_id')

#one-hot encoding for top5 categories
catList = []
businesses['categories'].fillna(value='',inplace=True)
businesses['cat'] = businesses['categories'].apply(lambda x: x.split(','))
catList.extend(businesses['cat'])
merged = [x.strip() for x in list(itertools.chain(*catList))]

#Adding state, review_count, is_open
base_df = base_df.merge(businesses[['business_id','state','review_count','is_open', 'hours', 'cat']], on='business_id')
base_df.rename(columns={"review_count": "business_review_count"}, inplace=True)

#getting top 5 common categories items
top5List = [x for x in list(itertools.chain(*Counter(merged).most_common(5))) if type(x) != int]

#one-hot encoding if the business in top 5 common category
for item in top5List:
    base_df[item] = base_df['cat'].apply(lambda categories: 'Y' if bool(set([y.strip() for y in categories]).intersection([item])) else 'N')
    #base_df[item] = base_df['cat'].apply(lambda categories: 'Y' if bool(set([y.strip() for y in categories]).intersection(set(item))) else 'N')

# is_open (categorical) change from 1 and 0 to Y and N
base_df['is_open'] = base_df['is_open'].apply(lambda x: 'Y' if x else 'N')

# hours: how many days per week it's open
base_df['hours'] = base_df['hours'].apply(lambda x: len(x.keys()) if x else 0)
base_df.rename(columns={"hours": "days_per_week_open"}, inplace=True)

# user information
base_df = base_df.merge(users[['user_id', 'average_stars', 'review_count', 'friends']],  on='user_id')

# number_of_friends
base_df['friends'] = base_df['friends'].apply(lambda x: len(x.split(',')))
base_df.rename(columns={"friends": "number_of_friends", "review_count": "user_review_count", "average_stars": "average_user_rating"}, inplace=True)

#number of tips for popularity measure of business
business_numberOfTips = tips[['business_id','user_id']].groupby('business_id').count().reset_index()
business_numberOfTips.rename(columns={"user_id": "business_numberOfTips"}, inplace=True)
base_df = base_df.merge(business_numberOfTips, on='business_id')

#number of photo for popularity measure of business
business_numberOfPhotos = photos[['business_id','photo_id']].groupby('business_id').count().reset_index()
business_numberOfPhotos.rename(columns={"photo_id": "business_numberOfPhotos"}, inplace=True)
base_df = base_df.merge(business_numberOfPhotos, on='business_id')

In [31]:
base_df.head()

,user_id,business_id,rating,date,average_business_rating,state,business_review_count,is_open,days_per_week_open,cat,Restaurants,Shopping,Food,Home Services,Beauty & Spas,average_user_rating,user_review_count,number_of_friends,business_numberOfTips,business_numberOfPhotos
0,-nxh-PAw8TcDR2CwPOIdwA,L9pTWWAATj7HoLaRe6ZA1Q,3.0,2016-11-09 23:35:25,3.0,AZ,444,Y,6,"[Cheesesteaks, Sandwiches, Restaurants, Ame...",Y,N,N,N,N,4.55,11,14,76,2
1,-nxh-PAw8TcDR2CwPOIdwA,13KW8P5n1jAxxHB7Bl_obg,5.0,2017-05-28 08:15:37,4.0,AZ,415,Y,5,"[Sandwiches, Polish, Japanese, Gluten-Free,...",Y,N,N,N,N,4.55,11,14,49,6
2,8SgjU2fID9VqKQooFt1DIQ,13KW8P5n1jAxxHB7Bl_obg,3.0,2013-08-05 14:25:58,4.0,AZ,415,Y,5,"[Sandwiches, Polish, Japanese, Gluten-Free,...",Y,N,N,N,N,3.97,306,2607,49,6
3,-nxh-PAw8TcDR2CwPOIdwA,KEwpIojt2zYIIt47FTlGCA,5.0,2016-09-22 21:42:46,5.0,AZ,41,N,6,"[Lebanese, Restaurants, Middle Eastern, Med...",Y,N,N,N,N,4.55,11,14,8,3
4,-nxh-PAw8TcDR2CwPOIdwA,i066yR2IDP4FWt6p-k9aFg,5.0,2015-10-24 23:34:06,5.0,AZ,454,Y,7,"[Sandwiches, Restaurants, Vegan, Gluten-Fre...",Y,N,N,N,N,4.55,11,14,112,3


# 4. Evaluation <a class="anchor" id="4"></a>

## 4.1 Accuracy <a class="anchor" id="4-1"></a>

## 4.2 Coverage <a class="anchor" id="4-2"></a>

# 5. Conclusion <a class="anchor" id="5"></a>